In [4]:
import requests
import json
from random import choice
from bs4 import BeautifulSoup
import csv

product_obj = {'is_active':False, 
           'is_discount':is_discount,
           'current_review_rating':0,
           'product_source':'HOMEPAGE',
              'product_link':product_link,
           'store':store,
         'name':name, 
           'description':description,
           'product_image_type': 'MP',
           'product_thumbnail_image':product_thumbnail_image,
           'video_source':None,
           'original_price':original_price,
           'discount_price':discount_price,
           'discount_rate':discount_rate,
           'currency':'VND',
           'is_free_ship':False,
           'stock':stock,
           'shopee_color':shopee_color_list,
           'shopee_size':[],
           'shopee_category':[],
           'size_chart':None,
               'productOption':product_option_list
          }
product_option_obj = {
           'is_active':True, 
           'name':name, 
           'original_price':original_price,
           'discount_price':discount_price,
           'currency':'VND',
           'stock':stock,
}

NameError: name 'is_discount' is not defined

In [5]:


_user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.38 (KHTML, like Gecko) Version/11.0 Mobile/15A372 Safari/604.1',
    'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 8.0.0; SM-G960F Build/R16NW) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.84 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 6.0; HTC One X10 Build/MRA58K; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/61.0.3163.98 Mobile Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.38 (KHTML, like Gecko) Version/11.0 Mobile/15A372 Safari/604.1',
    'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9',
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1'
]


def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    proxy_list = soup.find('tbody')
    proxies = set()
    for rows in proxy_list.find_all('tr')[:5]:
        proxies.add(rows.find_all('td')[0].text +
                    ':'+rows.find_all('td')[1].text)
    return proxies


class HomepageCrawler:
    def __init__(self, user_agents=None, proxy=None):
        self.user_agents = user_agents
        self.proxy = proxy

    def __random_agent(self):
        if self.user_agents and isinstance(self.user_agents, list):
            return choice(self.user_agents)
        return choice(_user_agents)

    def __random_proxies(self):
        if self.proxy and isinstance(self.proxy, list):
            # print('execute')
            return choice(self.proxy)
        proxies = get_proxies()
        return random.sample(get_proxies(), 1)[0]

    def request_url(self, url):
        try:
            response = requests.get(url,
                                    headers={'User-Agent': self.__random_agent(),
                                             'X-Requested-With': 'XMLHttpRequest',
                                             },
                                    proxies={'http': self.proxy, 'https': self.proxy})
            response.raise_for_status()
        except requests.HTTPError as e:
            # print(e)
            pass
        except requests.RequestException:
            raise requests.RequestException
        else:
            return response
    
#     @staticmethod
def extract_json_data(html):
    soup = BeautifulSoup(html, 'html.parser')
    body = soup.find('body')
    script_tag = body.find('script')
    raw_string = script_tag.text.strip().replace(
        'window._sharedData =', '').replace(';', '')
    return json.loads(raw_string)


In [21]:
def degrey_sub_crawler(product_source):
    product_list = []
    store = 291
    for product_obj in product_source:
    #     thumbnail / link
        product_link_soup = product_obj.find("a", href=True)
        product_link = "https://degrey.vn"+product_link_soup.get('href')
        product_thumbnail_image = "http:"+product_link_soup.find("img",class_="img-loop").get('src')
        print(product_link)

    #     title
        response = obj.request_url(product_link)
        product_soup =  BeautifulSoup(response.text,'html.parser')
        name = product_soup.find('h1').text
        print(name)
        description = product_soup.find('h1').text

    #     product image list
        product_image_list = product_soup.find_all('img', class_='product-image-feature')
        for product_image_obj in product_image_list:
            source = "http:" + product_image_obj.get('src')
            # print(source)

    #     product price
        product_price = product_soup.find('div',class_='product-price')
        if product_price.find('span',class_='pro-sale'):
            is_discount= True
            original_price = product_price.find('del').text
            discount_price = product_price.find('span',class_='pro-price').text
            discount_rate = product_price.find('span',class_='pro-sale').text
        else:
            original_price = product_price.find('span',class_='pro-price').text
            discount_price = None
            discount_rate = None
            is_discount= False

    #     product stock
        product_stock_soup = product_soup.find('button',class_='disabled add-to-cartProduct')
        if product_stock_soup:
            stock = 0
        else:
            stock = 9999

    #     product options
        product_options = product_soup.find('select').find_all('option')
        product_option_list = []
        for product_option in product_options:
            option_name = product_option.text
            product_option_obj = {
               'is_active':True, 
               'name':option_name, 
               'original_price':original_price,
               'discount_price':discount_price,
               'currency':'VND',
               'stock':stock, }        


    #     product color
        product_color_list = product_soup.find('div', class_='header')
        shopee_color_list = []
        if product_color_list:
            if product_color_list.text == "Kích thước:":
                product_color_list = product_color_list.parent.find_all('div',class_='n-sd')
                for product_color_obj in product_color_list:
                    display_name = product_color_obj.find('span').text
                    shopee_color_list.append({'display_name':display_name})
            else:
                print(product_color_list.text)
                break

        product_obj = {'is_active':False, 
                   'is_discount':is_discount,
                   'current_review_rating':0,
                   'product_source':'HOMEPAGE',
                      'product_link':product_link,
                   'store':store,
                 'name':name, 
                   'description':description,
                   'product_image_type': 'MP',
                   'product_thumbnail_image':product_thumbnail_image,
                   'video_source':None,
                   'original_price':original_price,
                   'discount_price':discount_price,
                   'discount_rate':discount_rate,
                   'currency':'VND',
                   'is_free_ship':False,
                   'stock':stock,
                   'shopee_color':shopee_color_list,
                   'shopee_size':[],
                   'shopee_category':[],
                   'size_chart':None,
                       'productOption':product_option_list
                  }
        product_list.append(product_obj)
    return product_list

def degrey_crawler():
    obj = HomepageCrawler()
    i = 1
    product_list = []
    while True:
        url = "https://degrey.vn/collections/tat-ca-san-pham?page=" + str(i)
        response = obj.request_url(url)
        soup = BeautifulSoup(response.text,'html.parser')
        product_source = soup.find_all("div", class_="product-block")
        product_sub_list = degrey_sub_crawler(product_source)
        product_list = product_list + product_sub_list
        length = len(product_source)
        print(length)
        if length == 0:
            break
        i= i+1
    return product_list

In [22]:
if __name__ == '__main__':
    print('start scrapying')
    product_list = degrey_crawler()


start scrapying
https://degrey.vn/products/skull-shirt-black-ss-black
Skull Shirt Black - SS Black
https://degrey.vn/products/skull-shirt-white-ss-white
Skull Shirt White - SS White
https://degrey.vn/products/thunder-shirt-ts
Thunder Shirt - TS
https://degrey.vn/products/fire-shirt-fs
Fire Shirt - FS
https://degrey.vn/products/blur-shirt-bs
Blur Shirt - BS
https://degrey.vn/products/astronaut-backpack-ab
Astronaut Backpack - AB
https://degrey.vn/products/signature-tee-snt
Signature Tee - SNT
https://degrey.vn/products/barbed-wire-shirt-bws
Barbed Wire Shirt - BWS
https://degrey.vn/products/magic-jacket-mj
Magic Jacket - MJ
https://degrey.vn/products/tui-banh-mi-tbm
Túi Bánh Mì - TBM
https://degrey.vn/products/flame-box-5-mask-flb
Flame Box(5 Mask) - FLB
https://degrey.vn/products/color-block-tee-cbt
Color Block Tee - CBT
https://degrey.vn/products/polyester-short-ps
Polyester Short - PS
https://degrey.vn/products/warning-longsleeve-den-wls-den
Warning LongSleeve Đen - WLS Đen
https://d

Future Tee Đỏ - FT Đỏ
https://degrey.vn/products/future-tee-den-ft-den
Future Tee Đen - FT Đen
https://degrey.vn/products/future-tee-vang-ft-vang
Future Tee Vàng - FT Vàng
https://degrey.vn/products/degrey-backpack-ss4-db-ss4
Degrey BackPack ss4 - DB ss4
https://degrey.vn/products/warning-tee-yellow-wt-va-ng
Warning Tee YELLOW - WT VÀNG
https://degrey.vn/products/degrey-belt-bag-dbb-kem
Degrey Belt Bag - DBB kem
https://degrey.vn/products/degrey-belt-bag-dbb-den
Degrey Belt Bag - DBB đen
https://degrey.vn/products/degrey-basic-cap-green-dbc-xanh-la
Degrey Basic Cap GREEN- DBC XANH LÁ
https://degrey.vn/products/degrey-basic-cap-kem-dbc-kem
Degrey Basic Cap WHITE - DBC TRẮNG KEM
https://degrey.vn/products/degrey-basic-cap-blue-navy-dbc-xanh-navy
Degrey Basic Cap BLUE NAVY - DBC XANH NAVY
https://degrey.vn/products/degrey-basic-cap-blakc-dbc-den
Degrey Basic Cap BLACK - DBC ĐEN
https://degrey.vn/products/small-shoulder-bag-ssb
Small Shoulder Bag - SSB
https://degrey.vn/products/daddy-shir

Xoài Magic TShirt Đen - XMT Đen
https://degrey.vn/products/xoai-magic-tshirt-trang-xmt-trang
Xoài Magic TShirt Trắng - XMT Trắng
https://degrey.vn/products/fire-tshirt-fts
Fire TShirt - FTS
https://degrey.vn/products/be-xoai-wallet-bxw
Bé Xoài Wallet - BXW
https://degrey.vn/products/dastel-vest-hong-dv-hong
Dastel Vest hồng - DV hồng
https://degrey.vn/products/degrey-sport-bag-dsb
Degrey Sport Bag - DSB
https://degrey.vn/products/regular-backpacks-pink-rgb-hong
REGULAR BACKPACKS PINK - RGB HỒNG
https://degrey.vn/products/regular-backpacks-blue-rgb-xanh-duong
REGULAR BACKPACKS BLUE - RGB XANH DƯƠNG
https://degrey.vn/products/regular-backpacks-green-rgb-xanh-chuoi
REGULAR BACKPACKS GREEN - RGB XANH CHUỐI
https://degrey.vn/products/regular-backpacks-red-rgb-do
REGULAR BACKPACKS RED - RGB ĐỎ
https://degrey.vn/products/invi-bag-purple-inb-tim
INVI BAG PURPLE - INB TÍM
https://degrey.vn/products/invi-bag-blue-inb-xanh-duong
INVI BAG BLUE - INB XANH DƯƠNG
https://degrey.vn/products/regular-te

REGULAR TEE - RT Tím
https://degrey.vn/products/adventure-time-tshirt-green-attg
ADVENTURE TIME TSHIRT GREEN - ATTG
[{'is_active': False, 'is_discount': False, 'current_review_rating': 0, 'product_source': 'HOMEPAGE', 'product_link': 'https://degrey.vn/products/regular-tee-rt-tim', 'store': 291, 'name': 'REGULAR TEE - RT Tím', 'description': 'REGULAR TEE - RT Tím', 'product_image_type': 'MP', 'product_thumbnail_image': 'http://product.hstatic.net/1000281824/product/5_5_2986f42855504fafb7dbe59f0d6bd4fd_grande.jpg', 'video_source': None, 'original_price': '300,000₫', 'discount_price': None, 'discount_rate': None, 'currency': 'VND', 'is_free_ship': False, 'stock': 9999, 'shopee_color': [{'display_name': 'NHỎ'}, {'display_name': 'TRUNG'}, {'display_name': 'LỚN'}], 'shopee_size': [], 'shopee_category': [], 'size_chart': None, 'productOption': []}, {'is_active': False, 'is_discount': True, 'current_review_rating': 0, 'product_source': 'HOMEPAGE', 'product_link': 'https://degrey.vn/products/a